# FinBERT LoRA Hyperparameter Sweep

This notebook extends the standard FinBERT sweep to tune **LoRA-specific** hyperparameters:
- LoRA rank (`lora_r`)
- LoRA alpha (`lora_alpha`) 
- LoRA dropout (`lora_dropout`)
- Target modules for LoRA adaptation
- Higher learning rates suitable for LoRA training

## Key Differences from Standard Sweep
- `discriminate=False` and `gradual_unfreeze=False` are fixed (incompatible with LoRA)
- Learning rate range is higher (1e-4 to 1e-3 vs 1e-5 to 5e-5)
- Additional LoRA-specific hyperparameters in sweep space

## Prerequisites
```bash
pip install wandb peft
wandb login
```


## 1. Setup and Imports


In [21]:
from pathlib import Path
import shutil
import os
import logging
import sys
import numpy as np
sys.path.append('..')

from sklearn.metrics import classification_report, f1_score
from transformers import AutoModelForSequenceClassification
import torch
from torch.nn import CrossEntropyLoss, MSELoss

from finbert.finbert import *
import finbert.utils as tools

# Weights & Biases
import wandb

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

print("Imports loaded successfully")
print(f"Project directory: {project_dir}")


Imports loaded successfully
Project directory: /home/si2449/hpml-project/pipelines/finBERT


## 2. Configuration

Set up paths and W&B project name.


In [ ]:
# Paths
cl_path = project_dir/'models'/'sentiment_lora'
cl_data_path = project_dir/'data'/'sentiment_data'

# W&B Configuration
WANDB_PROJECT = "Project-Runs"
WANDB_ENTITY = None  # Set your W&B entity/username if needed

print(f"Model path: {cl_path}")
print(f"Data path: {cl_data_path}")
print(f"W&B Project: {WANDB_PROJECT}")


Model path: /home/si2449/hpml-project/pipelines/finBERT/models/sentiment_lora
Data path: /home/si2449/hpml-project/pipelines/finBERT/data/sentiment_data
W&B Project: finbert-lora-hyperparameter-sweep


## 3. Define LoRA Sweep Configuration

This defines the hyperparameter search space specifically for LoRA fine-tuning.

### Key LoRA Parameters:
- **lora_r**: Rank of the low-rank matrices. Higher = more parameters, more capacity
- **lora_alpha**: Scaling factor. Often set to 2*r or equal to r
- **lora_dropout**: Dropout applied to LoRA layers
- **lora_target_modules**: Which attention matrices to apply LoRA to


In [23]:
sweep_config = {
    'method': 'bayes',  # 'grid', 'random', or 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        # LoRA-specific parameters
        'lora_r': {
            'values': [4, 8, 16, 32]
        },
        'lora_alpha': {
            'values': [8, 16, 32, 64]
        },
        'lora_dropout': {
            'distribution': 'uniform',
            'min': 0.0,
            'max': 0.2
        },
        'lora_target_modules': {
            'values': [
                ['query', 'value'],           # Standard: Q and V matrices
                ['query', 'key', 'value'],    # All attention matrices
                ['query', 'value', 'dense'],  # Q, V + output projection
            ]
        },
        # Higher learning rates for LoRA (key difference from full fine-tuning)
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 1e-3
        },
        # Standard training parameters
        'num_train_epochs': {
            'values': [5, 8, 10, 15]
        },
        'train_batch_size': {
            'values': [16, 32, 64]
        },
        'warm_up_proportion': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.3
        },
        'max_seq_length': {
            'values': [48, 64, 96]
        }
    }
}

print("LoRA Sweep configuration created")
print(f"  Method: {sweep_config['method']}")
print(f"  Optimization metric: {sweep_config['metric']['name']}")
print(f"  LoRA parameters being tuned:")
print(f"    - lora_r: {sweep_config['parameters']['lora_r']['values']}")
print(f"    - lora_alpha: {sweep_config['parameters']['lora_alpha']['values']}")
print(f"    - lora_dropout: [{sweep_config['parameters']['lora_dropout']['min']}, {sweep_config['parameters']['lora_dropout']['max']}]")


LoRA Sweep configuration created
  Method: bayes
  Optimization metric: val_loss
  LoRA parameters being tuned:
    - lora_r: [4, 8, 16, 32]
    - lora_alpha: [8, 16, 32, 64]
    - lora_dropout: [0.0, 0.2]


## 4. Training Function with LoRA and W&B Integration

This wraps the LoRA-enabled training code with W&B logging.


In [24]:
def train_with_lora_config(config=None):
    """
    Training function that W&B will call with different LoRA hyperparameters.
    """
    # Initialize W&B run
    with wandb.init(config=config):
        # Get hyperparameters from W&B
        config = wandb.config
        
        print(f"\n{'='*80}")
        print(f"Starting LoRA training run with config:")
        print(f"  LoRA r: {config.lora_r}")
        print(f"  LoRA alpha: {config.lora_alpha}")
        print(f"  LoRA dropout: {config.lora_dropout:.4f}")
        print(f"  LoRA target modules: {config.lora_target_modules}")
        print(f"  Learning rate: {config.learning_rate:.6f}")
        print(f"  Epochs: {config.num_train_epochs}")
        print(f"  Batch size: {config.train_batch_size}")
        print(f"  Warmup: {config.warm_up_proportion:.4f}")
        print(f"  Max seq length: {config.max_seq_length}")
        print(f"{'='*80}\n")
        
        # Clean previous model directory
        model_path = project_dir / 'models' / 'lora_sweep' / f'sweep_{wandb.run.id}'
        try:
            shutil.rmtree(model_path)
        except:
            pass
        
        # Create BERT model
        bertmodel = AutoModelForSequenceClassification.from_pretrained(
            'bert-base-uncased', cache_dir=None, num_labels=3
        )
        
        # Create FinBERT config with LoRA hyperparameters from sweep
        finbert_config = Config(
            data_dir=cl_data_path,
            bert_model=bertmodel,
            num_train_epochs=config.num_train_epochs,
            model_dir=model_path,
            max_seq_length=config.max_seq_length,
            train_batch_size=config.train_batch_size,
            learning_rate=config.learning_rate,
            output_mode='classification',
            warm_up_proportion=config.warm_up_proportion,
            local_rank=-1,
            # LoRA settings
            use_lora=True,
            lora_r=config.lora_r,
            lora_alpha=config.lora_alpha,
            lora_dropout=config.lora_dropout,
            lora_target_modules=tuple(config.lora_target_modules),
            # These must be OFF for LoRA
            discriminate=False,
            gradual_unfreeze=False
        )
        
        # Initialize FinBERT
        finbert = FinBert(finbert_config)
        finbert.base_model = 'bert-base-uncased'
        finbert.prepare_model(label_list=['positive', 'negative', 'neutral'])
        
        # Load data
        train_data = finbert.get_data('train')
        test_data = finbert.get_data('test')
        
        # Create model with LoRA
        model = finbert.create_the_model()
        
        # Log trainable parameters info
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        wandb.log({
            'total_params': total_params,
            'trainable_params': trainable_params,
            'trainable_percent': 100 * trainable_params / total_params
        })
        
        # Train with W&B logging
        trained_model = train_lora_with_wandb_logging(
            finbert, train_data, model, test_data
        )
        
        # Final evaluation
        results = finbert.evaluate(examples=test_data, model=trained_model)
        results['prediction'] = results.predictions.apply(lambda x: np.argmax(x, axis=0))
        
        # Calculate metrics
        metrics = calculate_metrics(results, finbert)
        
        # Log final metrics to W&B
        wandb.log({
            'final_test_loss': metrics['loss'],
            'final_test_accuracy': metrics['accuracy'],
            'final_f1_positive': metrics['f1_positive'],
            'final_f1_negative': metrics['f1_negative'],
            'final_f1_neutral': metrics['f1_neutral'],
            'final_f1_macro': metrics['f1_macro']
        })
        
        print(f"\n{'='*80}")
        print(f"Final Results:")
        print(f"  Test Loss: {metrics['loss']:.4f}")
        print(f"  Test Accuracy: {metrics['accuracy']:.4f}")
        print(f"  Macro F1: {metrics['f1_macro']:.4f}")
        print(f"  Trainable params: {trainable_params:,} ({100*trainable_params/total_params:.2f}%)")
        print(f"{'='*80}\n")
        
        return metrics


print("Main training function defined")


Main training function defined


In [25]:
def train_lora_with_wandb_logging(finbert, train_data, model, test_data):
    """
    Modified training loop with W&B logging for LoRA models.
    Note: gradual_unfreeze is disabled for LoRA training.
    """
    validation_examples = finbert.get_data('validation')
    global_step = 0
    finbert.validation_losses = []
    
    train_dataloader = finbert.get_loader(train_data, 'train') 
    model.train()
    step_number = len(train_dataloader)
    
    best_val_loss = float('inf')
    best_model_epoch = 0
    
    for epoch in trange(int(finbert.config.num_train_epochs), desc="Epoch"):
        model.train()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        
        for step, batch in enumerate(tqdm(train_dataloader, desc='Iteration')):
            batch = tuple(t.to(finbert.device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, agree_ids = batch
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            logits = outputs.logits
            weights = finbert.class_weights.to(finbert.device)
            
            if finbert.config.output_mode == "classification":
                loss_fct = CrossEntropyLoss(weight=weights)
                loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))
            elif finbert.config.output_mode == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), label_ids.view(-1))
            
            if finbert.config.gradient_accumulation_steps > 1:
                loss = loss / finbert.config.gradient_accumulation_steps
            else:
                loss.backward()
            
            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            
            if (step + 1) % finbert.config.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(
                    (p for p in model.parameters() if p.requires_grad), 1.0
                )
                finbert.optimizer.step()
                finbert.scheduler.step()
                finbert.optimizer.zero_grad()
                global_step += 1
                
                # Log to W&B every N steps
                if global_step % 10 == 0:
                    wandb.log({
                        'train_loss': tr_loss / nb_tr_steps,
                        'learning_rate': finbert.optimizer.param_groups[0]['lr'],
                        'epoch': epoch,
                        'step': global_step
                    })
        
        # Validation at end of epoch
        validation_loader = finbert.get_loader(validation_examples, 'eval')
        model.eval()
        
        valid_loss, valid_accuracy = 0, 0
        nb_valid_steps, nb_valid_examples = 0, 0
        
        for input_ids, attention_mask, token_type_ids, label_ids, agree_ids in tqdm(validation_loader, desc="Validating"):
            input_ids = input_ids.to(finbert.device)
            attention_mask = attention_mask.to(finbert.device)
            token_type_ids = token_type_ids.to(finbert.device)
            label_ids = label_ids.to(finbert.device)
            agree_ids = agree_ids.to(finbert.device)
            
            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                )
                logits = outputs.logits
                
                if finbert.config.output_mode == "classification":
                    loss_fct = CrossEntropyLoss(weight=weights)
                    tmp_valid_loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))
                elif finbert.config.output_mode == "regression":
                    loss_fct = MSELoss()
                    tmp_valid_loss = loss_fct(logits.view(-1), label_ids.view(-1))
                
                valid_loss += tmp_valid_loss.mean().item()
                nb_valid_steps += 1
        
        valid_loss = valid_loss / nb_valid_steps
        finbert.validation_losses.append(valid_loss)
        
        # Log validation metrics to W&B
        wandb.log({
            'val_loss': valid_loss,
            'epoch': epoch,
            'best_val_loss': min(finbert.validation_losses)
        })
        
        print(f"Epoch {epoch}: Validation loss = {valid_loss:.4f}")
        
        # Save best model
        if valid_loss == min(finbert.validation_losses):
            try:
                os.remove(finbert.config.model_dir / ('temporary' + str(best_model_epoch)))
            except:
                pass
            torch.save({'epoch': str(epoch), 'state_dict': model.state_dict()},
                       finbert.config.model_dir / ('temporary' + str(epoch)))
            best_model_epoch = epoch
            best_val_loss = valid_loss
    
    # Load best model
    checkpoint = torch.load(finbert.config.model_dir / ('temporary' + str(best_model_epoch)))
    model.load_state_dict(checkpoint['state_dict'])
    
    # Save final model
    model_to_save = model.module if hasattr(model, 'module') else model
    output_model_file = os.path.join(finbert.config.model_dir, 'pytorch_model.bin')
    torch.save(model_to_save.state_dict(), output_model_file)
    
    # Clean up temporary files
    try:
        os.remove(finbert.config.model_dir / ('temporary' + str(best_model_epoch)))
    except:
        pass
    
    return model


def calculate_metrics(results, finbert):
    """
    Calculate comprehensive metrics for evaluation.
    """
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(
        torch.tensor(list(results['predictions'])),
        torch.tensor(list(results['labels']))
    )
    
    accuracy = (results['labels'] == results['prediction']).sum() / results.shape[0]
    
    # Calculate per-class F1 scores
    f1_scores = f1_score(results['labels'], results['prediction'], average=None)
    f1_macro = f1_score(results['labels'], results['prediction'], average='macro')
    
    return {
        'loss': loss.item(),
        'accuracy': accuracy,
        'f1_positive': f1_scores[0],
        'f1_negative': f1_scores[1],
        'f1_neutral': f1_scores[2],
        'f1_macro': f1_macro
    }


print("Helper functions defined")


Helper functions defined


## 5. Initialize and Run LoRA Sweep

This will start the hyperparameter sweep. W&B will automatically try different LoRA configurations.


In [26]:
# Initialize the sweep
sweep_id = wandb.sweep(
    sweep_config, 
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY
)

print(f"LoRA Sweep initialized with ID: {sweep_id}")
print(f"View at: https://wandb.ai/{WANDB_ENTITY or 'your-username'}/{WANDB_PROJECT}/sweeps/{sweep_id}")


Create sweep with ID: wgeew1ms
Sweep URL: https://wandb.ai/si2449-columbia-university/finbert-lora-hyperparameter-sweep/sweeps/wgeew1ms
LoRA Sweep initialized with ID: wgeew1ms
View at: https://wandb.ai/your-username/finbert-lora-hyperparameter-sweep/sweeps/wgeew1ms


In [27]:
# Run the sweep
# count: number of runs to execute (increase for more thorough search)
wandb.agent(sweep_id, function=train_with_lora_config, count=15)

print("\n" + "="*80)
print("LORA SWEEP COMPLETED")
print("="*80)
print(f"View results at: https://wandb.ai/{WANDB_ENTITY or 'your-username'}/{WANDB_PROJECT}/sweeps/{sweep_id}")


wandb: Agent Starting Run: wai0o21b with config:
wandb: 	learning_rate: 0.0009999359926900096
wandb: 	lora_alpha: 32
wandb: 	lora_dropout: 0.043564732222302394
wandb: 	lora_r: 32
wandb: 	lora_target_modules: ['query', 'value']
wandb: 	max_seq_length: 64
wandb: 	num_train_epochs: 8
wandb: 	train_batch_size: 64
wandb: 	warm_up_proportion: 0.1021149770831856



Starting LoRA training run with config:
  LoRA r: 32
  LoRA alpha: 32
  LoRA dropout: 0.0436
  LoRA target modules: ['query', 'value']
  Learning rate: 0.001000
  Epochs: 8
  Batch size: 64
  Warmup: 0.1021
  Max seq length: 64



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/17/2025 06:17:09 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
12/17/2025 06:17:10 - INFO - finbert.utils -   *** Example ***
12/17/2025 06:17:10 - INFO - finbert.utils -   guid: train-1
12/17/2025 06:17:10 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/17/2025 06:17:10 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 2723

trainable params: 1,181,955 || all params: 110,666,502 || trainable%: 1.0680


12/17/2025 06:17:11 - INFO - finbert.finbert -   ***** Loading data *****
12/17/2025 06:17:11 - INFO - finbert.finbert -     Num examples = 3488
12/17/2025 06:17:11 - INFO - finbert.finbert -     Batch size = 64
12/17/2025 06:17:11 - INFO - finbert.finbert -     Num steps = 48
Iteration: 100%|██████████| 55/55 [00:24<00:00,  2.24it/s]
12/17/2025 06:17:36 - INFO - finbert.utils -   *** Example ***
12/17/2025 06:17:36 - INFO - finbert.utils -   guid: validation-1
12/17/2025 06:17:36 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/17/2025 06:17:36 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 06:17:36 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

...

Traceback (most recent call last):
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 967, in save
    _save(
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 1268, in _save
    zip_file.write_record(name, storage, num_bytes)
RuntimeError: [enforce fail at inline_container.cc:858] . PytorchStreamWriter failed writing file data/263: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/tmp/ipykernel_3836044/1136645875.py", line 80, in train_with_lora_config
    trained_model = train_lora_with_wandb_logging(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/tmp/ipykernel_3836044/1780291689.py", line 130, in train_lora_with_wandb_logging
    torch.save(model_to_save.state_dict(), output_model_file)
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 966, 

best_val_loss,█▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
learning_rate,▆█▇▇▆▅▄▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
total_params,▁
train_loss,██▇▇▆▃▃▃▃▃▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainable_params,▁
trainable_percent,▁
val_loss,█▁▁▁▁▁▁▁
best_val_loss,0.52118
epoch,7


--- Logging error ---
Traceback (most recent call last):
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 967, in save
    _save(
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 1268, in _save
    zip_file.write_record(name, storage, num_bytes)
RuntimeError: [enforce fail at inline_container.cc:858] . PytorchStreamWriter failed writing file data/263: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/tmp/ipykernel_3836044/1136645875.py", line 80, in train_with_lora_config
    trained_model = train_lora_with_wandb_logging(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/tmp/ipykernel_3836044/1780291689.py", line 130, in train_lora_with_wandb_logging
    torch.save(model_to_save.state_dict(), output_model_file)
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/seriali

Traceback (most recent call last):
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 967, in save
    _save(
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 1268, in _save
    zip_file.write_record(name, storage, num_bytes)
RuntimeError: [enforce fail at inline_container.cc:858] . PytorchStreamWriter failed writing file data/263: file write failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/tmp/ipykernel_3836044/1136645875.py", line 80, in train_with_lora_config
    trained_model = train_lora_with_wandb_logging(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/tmp/ipykernel_3836044/1780291689.py", line 130, in train_lora_with_wandb_logging
    torch.save(model_to_save.state_dict(), output_model_file)
  File "/home/si2449/hpml-project/venv/lib/python3.11/site-packages/torch/serialization.py", line 966, 


LORA SWEEP COMPLETED
View results at: https://wandb.ai/your-username/finbert-lora-hyperparameter-sweep/sweeps/wgeew1ms


In [1]:
def train_with_wandb_logging(finbert, train_data, model, use_lora: bool):
    validation_examples = finbert.get_data("validation")
    train_dataloader = finbert.get_loader(train_data, "train")

    build_optimizer_and_scheduler(model, finbert, train_dataloader)

    global_step = 0
    finbert.validation_losses = []

    step_number = len(train_dataloader)
    i = 0
    best_model = None

    for epoch in trange(int(finbert.config.num_train_epochs), desc="Epoch"):
        model.train()
        tr_loss = 0.0
        nb_tr_steps = 0

        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            if finbert.config.gradual_unfreeze and (not use_lora) and i == 0:
                backbone = get_bert_backbone(model)
                for param in backbone.parameters():
                    param.requires_grad = False

            if (step_number // 3) > 0 and (step % (step_number // 3)) == 0:
                i += 1

            if finbert.config.gradual_unfreeze and (not use_lora):
                backbone = get_bert_backbone(model)
                enc = backbone.encoder.layer
                enc_no = finbert.config.encoder_no

                if i > 1 and i < enc_no:
                    for k in range(i - 1):
                        for param in enc[enc_no - 1 - k].parameters():
                            param.requires_grad = True

                if i > enc_no + 1:
                    for param in backbone.embeddings.parameters():
                        param.requires_grad = True

            batch = tuple(t.to(finbert.device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, agree_ids = batch

            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            logits = outputs[0] if isinstance(outputs, (tuple, list)) else outputs.logits

            weights = finbert.class_weights.to(finbert.device)
            loss_fct = CrossEntropyLoss(weight=weights)
            loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))

            grad_acc = getattr(finbert.config, "gradient_accumulation_steps", 1) or 1
            loss = loss / grad_acc
            loss.backward()

            tr_loss += loss.item()
            nb_tr_steps += 1

            if (step + 1) % grad_acc == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                finbert.optimizer.step()
                finbert.scheduler.step()
                finbert.optimizer.zero_grad()
                global_step += 1

                if global_step % 10 == 0:
                    wandb.log({
                        "train_loss": tr_loss / max(nb_tr_steps, 1),
                        "learning_rate": finbert.optimizer.param_groups[0]["lr"],
                        "epoch": epoch,
                        "step": global_step,
                    })

        validation_loader = finbert.get_loader(validation_examples, "eval")
        model.eval()

        valid_loss = 0.0
        nb_valid_steps = 0

        with torch.no_grad():
            for input_ids, attention_mask, token_type_ids, label_ids, agree_ids in tqdm(validation_loader, desc="Validating"):
                input_ids = input_ids.to(finbert.device)
                attention_mask = attention_mask.to(finbert.device)
                token_type_ids = token_type_ids.to(finbert.device)
                label_ids = label_ids.to(finbert.device)

                outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                logits = outputs[0] if isinstance(outputs, (tuple, list)) else outputs.logits

                tmp_valid_loss = loss_fct(logits.view(-1, finbert.num_labels), label_ids.view(-1))
                valid_loss += float(tmp_valid_loss.item())
                nb_valid_steps += 1

        valid_loss /= max(nb_valid_steps, 1)
        finbert.validation_losses.append(valid_loss)

        wandb.log({
            "val_loss": valid_loss,
            "epoch": epoch,
            "best_val_loss": min(finbert.validation_losses),
        })

        print(f"Epoch {epoch}: Validation loss = {valid_loss:.4f}")

        if valid_loss == min(finbert.validation_losses):
            best_model = epoch
            os.makedirs(finbert.config.model_dir, exist_ok=True)
            torch.save({"epoch": int(epoch), "state_dict": model.state_dict()},
                       Path(finbert.config.model_dir) / f"temporary_{epoch}.pt")

    if best_model is not None:
        ckpt = torch.load(Path(finbert.config.model_dir) / f"temporary_{best_model}.pt", map_location=finbert.device)
        model.load_state_dict(ckpt["state_dict"])

    return model

import math
import wandb

def train_with_config():
    cfg = wandb.config
    finbert = FinBert(cfg)

    # --- build train loader FIRST so we can compute total optimizer steps ---
    train_data = finbert.get_data("train")
    train_dataloader = finbert.get_loader(train_data, "train")

    grad_acc = getattr(cfg, "gradient_accumulation_steps", 1) or 1
    steps_per_epoch = math.ceil(len(train_dataloader) / grad_acc)
    finbert.num_train_optimization_steps = steps_per_epoch * int(cfg.num_train_epochs)

    # --- now create base model (this was crashing before) ---
    model = finbert.create_the_model()

    # --- apply LoRA AFTER model exists ---
    if cfg.use_lora:
        from peft import LoraConfig, TaskType, get_peft_model

        target_modules = [s.strip() for s in str(cfg.lora_target).split(",") if s.strip()]

        lora_cfg = LoraConfig(
            r=int(cfg.lora_r),
            lora_alpha=int(cfg.lora_alpha),
            lora_dropout=float(cfg.lora_dropout),
            target_modules=target_modules,
            bias="none",
            task_type=TaskType.SEQ_CLS,
        )
        model = get_peft_model(model, lora_cfg)
        model.to(finbert.device)
        model.print_trainable_parameters()

    # train (you can reuse train_data; your train fn will rebuild loader internally)
    model = train_with_wandb_logging(finbert, train_data, model, use_lora=bool(cfg.use_lora))
    return model


## 6. Analyze Results

After the sweep completes, examine the best configuration from the W&B dashboard.

Key metrics to compare:
- **val_loss**: Primary optimization target
- **final_test_accuracy**: Test set performance
- **final_f1_macro**: Balanced F1 across classes
- **trainable_params**: Parameter efficiency


In [28]:
# Once you've identified the best config from the W&B dashboard, record it here:
# Example (update with your actual best values from the sweep)
best_lora_config = {
    # LoRA parameters
    'lora_r': 8,
    'lora_alpha': 16,
    'lora_dropout': 0.1,
    'lora_target_modules': ['query', 'value'],
    # Training parameters
    'learning_rate': 5e-4,
    'num_train_epochs': 10,
    'train_batch_size': 32,
    'warm_up_proportion': 0.2,
    'max_seq_length': 64,
}

print("Best LoRA configuration (update after sweep):")
for k, v in best_lora_config.items():
    print(f"  {k}: {v}")


Best LoRA configuration (update after sweep):
  lora_r: 8
  lora_alpha: 16
  lora_dropout: 0.1
  lora_target_modules: ['query', 'value']
  learning_rate: 0.0005
  num_train_epochs: 10
  train_batch_size: 32
  warm_up_proportion: 0.2
  max_seq_length: 64
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


## 7. Retrain with Best Config (Optional)

Use the best configuration to train a final model.


In [29]:
# Uncomment to retrain with best config
# train_with_lora_config(config=best_lora_config)
